<a href="https://colab.research.google.com/github/dsamithmendis/python-programming/blob/main/Copy_of_EEX3372_lab04_423646209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EEX3372 - PROGRAMMING IN PYTHON [ LAB – 04 ]

In [ ]:

"""sis.py - Student Information System library

Contains:
- Student class
- Course class
- Administrator class
- JSON save/load helpers
- A simple CLI function (does not run on import)

Usage:
- Import the classes and use in your own scripts, or run cli() to use the command-line interface:
    from sis import cli
    cli()

Notes:
- Data persistence uses JSON. Use Administrator.save_to_file(path) and load_from_file(path).
"""

from dataclasses import dataclass, field, asdict
from typing import Dict, List, Optional
import json
import datetime


@dataclass
class Student:
    """Representation of a student."""
    student_id: str
    name: str
    email: str
    enrolled_courses: List[str] = field(default_factory=list)
    grades: Dict[str, Optional[float]] = field(default_factory=dict)

    def display(self) -> str:
        """Return a human-readable summary of the student."""
        lines = [
            f"ID: {self.student_id}",
            f"Name: {self.name}",
            f"Email: {self.email}",
            f"Enrolled courses: {', '.join(self.enrolled_courses) if self.enrolled_courses else 'None'}",
            f"Grades: {self.grades if self.grades else 'None'}",
        ]
        return "\\n".join(lines)

    def enroll(self, course_id: str) -> None:
        """Enroll the student in a course (only record the id here)."""
        if course_id in self.enrolled_courses:
            raise ValueError(f"Student {self.student_id} already enrolled in course {course_id}.")
        self.enrolled_courses.append(course_id)
        # Initialize grade as None (not graded yet)
        self.grades[course_id] = None

    def view_enrolled_courses(self) -> List[str]:
        """Return list of enrolled course IDs."""
        return list(self.enrolled_courses)

    def to_dict(self) -> Dict:
        """Serialize student to dict for JSON storage."""
        return asdict(self)

    @staticmethod
    def from_dict(data: Dict) -> "Student":
        return Student(
            student_id=data["student_id"],
            name=data["name"],
            email=data["email"],
            enrolled_courses=data.get("enrolled_courses", []),
            grades=data.get("grades", {}),
        )


@dataclass
class Course:
    """Representation of a course."""
    course_id: str
    title: str
    capacity: int = 30
    roster: List[str] = field(default_factory=list)  # list of student IDs

    def display(self) -> str:
        lines = [
            f"Course ID: {self.course_id}",
            f"Title: {self.title}",
            f"Capacity: {self.capacity}",
            f"Enrolled: {len(self.roster)}",
            f"Roster: {', '.join(self.roster) if self.roster else 'None'}",
        ]
        return "\\n".join(lines)

    def add_student(self, student_id: str) -> None:
        """Add a student to the roster if there is space."""
        if student_id in self.roster:
            raise ValueError(f"Student {student_id} already in course {self.course_id}.")
        if len(self.roster) >= self.capacity:
            raise OverflowError(f"Course {self.course_id} is full (capacity {self.capacity}).")
        self.roster.append(student_id)

    def remove_student(self, student_id: str) -> None:
        """Remove a student from roster (if present)."""
        if student_id in self.roster:
            self.roster.remove(student_id)
        else:
            raise KeyError(f"Student {student_id} not found in course {self.course_id}.")

    def to_dict(self) -> Dict:
        return asdict(self)

    @staticmethod
    def from_dict(data: Dict) -> "Course":
        return Course(
            course_id=data["course_id"],
            title=data["title"],
            capacity=data.get("capacity", 30),
            roster=data.get("roster", []),
        )


class Administrator:
    """Administrator manages students and courses."""

    def __init__(self):
        self.students: Dict[str, Student] = {}
        self.courses: Dict[str, Course] = {}

    # Student management
    def add_student(self, student_id: str, name: str, email: str) -> Student:
        """Add a new student. student_id must be unique."""
        if student_id in self.students:
            raise KeyError(f"Student {student_id} already exists.")
        student = Student(student_id=student_id, name=name, email=email)
        self.students[student_id] = student
        return student

    def get_student(self, student_id: str) -> Student:
        if student_id not in self.students:
            raise KeyError(f"Student {student_id} not found.")
        return self.students[student_id]

    # Course management
    def add_course(self, course_id: str, title: str, capacity: int = 30) -> Course:
        if course_id in self.courses:
            raise KeyError(f"Course {course_id} already exists.")
        course = Course(course_id=course_id, title=title, capacity=capacity)
        self.courses[course_id] = course
        return course

    def get_course(self, course_id: str) -> Course:
        if course_id not in self.courses:
            raise KeyError(f"Course {course_id} not found.")
        return self.courses[course_id]

    # Enrollment
    def enroll_student_in_course(self, student_id: str, course_id: str) -> None:
        """Enroll a student in a course, updating both student and course structures."""
        student = self.get_student(student_id)
        course = self.get_course(course_id)

        # Attempt to add to course first (so capacity check occurs)
        course.add_student(student_id)
        try:
            student.enroll(course_id)
        except Exception as e:
            # Rollback course roster if student enroll fails
            course.remove_student(student_id)
            raise

    # Grading
    def grade_student(self, student_id: str, course_id: str, grade: float) -> None:
        """Assign a grade to a student for a course. Student must be enrolled."""
        student = self.get_student(student_id)
        if course_id not in student.enrolled_courses:
            raise KeyError(f"Student {student_id} is not enrolled in course {course_id}.")
        student.grades[course_id] = grade

    # Persistence
    def save_to_file(self, path: str) -> None:
        """Save the current state to a JSON file."""
        data = {
            "meta": {
                "saved_at": datetime.datetime.utcnow().isoformat() + "Z",
            },
            "students": {sid: s.to_dict() for sid, s in self.students.items()},
            "courses": {cid: c.to_dict() for cid, c in self.courses.items()},
        }
        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

    @staticmethod
    def load_from_file(path: str) -> "Administrator":
        """Load system state from a JSON file. Raises FileNotFoundError if path missing."""
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        admin = Administrator()
        for sid, sdata in data.get("students", {}).items():
            admin.students[sid] = Student.from_dict(sdata)
        for cid, cdata in data.get("courses", {}).items():
            admin.courses[cid] = Course.from_dict(cdata)
        return admin

    # Utilities for display
    def list_students(self) -> List[str]:
        return list(self.students.keys())

    def list_courses(self) -> List[str]:
        return list(self.courses.keys())


# Simple command-line interface (keeps logic separate so importing doesn't run it)
def cli():
    """Interactive CLI for administrators.

    Note: This function should be called explicitly. It is not executed on import.
    """
    import sys

    admin = Administrator()
    state_file = "sis_state.json"
    print("Student Information System - CLI")
    print("Type 'help' to see commands. Type 'exit' to quit.")
    while True:
        try:
            cmd = input("sis> ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\\nExiting...")
            break
        if not cmd:
            continue
        import shlex
        parts = shlex.split(cmd)
        action = parts[0].lower()

        try:
            if action == "help":
                print("""
Commands:
  add_student <id> <name> <email>
  add_course <course_id> <title> [capacity]
  enroll <student_id> <course_id>
  grade <student_id> <course_id> <grade>
  show_student <student_id>
  show_course <course_id>
  save [path]
  load [path]
  list_students
  list_courses
  exit
""")
            elif action == "add_student":
                _, sid, name, email = parts[:4]
                admin.add_student(sid, name, email)
                print(f"Added student {sid}")
            elif action == "add_course":
                if len(parts) < 3:
                    print("Usage: add_course <course_id> <title> [capacity]")
                    continue
                cid = parts[1]
                title = parts[2]
                capacity = int(parts[3]) if len(parts) > 3 else 30
                admin.add_course(cid, title, capacity)
                print(f"Added course {cid}")
            elif action == "enroll":
                _, sid, cid = parts[:3]
                admin.enroll_student_in_course(sid, cid)
                print(f"Enrolled {sid} in {cid}")
            elif action == "grade":
                _, sid, cid, grade_str = parts[:4]
                grade = float(grade_str)
                admin.grade_student(sid, cid, grade)
                print(f"Graded {sid} in {cid} : {grade}")
            elif action == "show_student":
                _, sid = parts[:2]
                print(admin.get_student(sid).display())
            elif action == "show_course":
                _, cid = parts[:2]
                print(admin.get_course(cid).display())
            elif action == "save":
                path = parts[1] if len(parts) > 1 else state_file
                admin.save_to_file(path)
                print(f"Saved state to {path}")
            elif action == "load":
                path = parts[1] if len(parts) > 1 else state_file
                admin = Administrator.load_from_file(path)
                print(f"Loaded state from {path}")
            elif action == "list_students":
                print(", ".join(admin.list_students()))
            elif action == "list_courses":
                print(", ".join(admin.list_courses()))
            elif action == "exit":
                print("Goodbye.")
                break
            else:
                print("Unknown command. Type 'help' for a list of commands.")
        except Exception as e:
            print(f"Error: {e}")


if __name__ == "__main__":
    cli()


Student Information System - CLI
Type 'help' to see commands. Type 'exit' to quit.
sis> help

Commands:
  add_student <id> <name> <email>
  add_course <course_id> <title> [capacity]
  enroll <student_id> <course_id>
  grade <student_id> <course_id> <grade>
  show_student <student_id>
  show_course <course_id>
  save [path]
  load [path]
  list_students
  list_courses
  exit

sis> add_student S001 "Alice Johnson" alice@example.com
Added student S001
sis> add_course C101 "Intro to Python" 30
Added course C101
sis> enroll S001 C101
Enrolled S001 in C101
sis> grade S001 C101 95
Graded S001 in C101 : 95.0
sis> show_student S001
ID: S001\nName: Alice Johnson\nEmail: alice@example.com\nEnrolled courses: C101\nGrades: {'C101': 95.0}
sis> show_course C101
Course ID: C101\nTitle: Intro to Python\nCapacity: 30\nEnrolled: 1\nRoster: S001
sis> exit
Goodbye.
